In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import tables            as tb
import random
import glob
import warnings
sns.set()

from  invisible_cities.io.dst_io              import load_dsts
from  invisible_cities.core .core_functions   import in_range
from  invisible_cities.core.system_of_units_c import units
import invisible_cities.reco.corrections      as corrf

from krcal.core.io_functions       import kdst_write
from krcal.core.io_functions       import filenames_from_list
from krcal.core.io_functions       import write_maps
from krcal.core.io_functions       import read_maps
from krcal.core.io_functions       import write_maps_ts
from krcal.core.io_functions       import read_maps_ts

from krcal.core.histo_functions    import h1, h1d, h2, h2d, plot_histo

from krcal.core.core_functions      import time_delta_from_time
from krcal.core.core_functions      import phirad_to_deg
from krcal.core.core_functions      import NN
from krcal.core.core_functions      import timeit
from krcal.core                     import fit_functions_ic as fitf

from krcal.core.plt_functions  import plot_sector
from krcal.core.plt_functions  import plot_sectors

from krcal.core.stat_functions      import mean_and_std

from krcal.core.analysis_functions  import kr_ranges_and_bins
from krcal.core.analysis_functions  import fiducial_volumes
from krcal.core.analysis_functions  import fid_eff
from krcal.core.analysis_functions  import select_rphi_sectors
from krcal.core.analysis_functions  import select_xy_sectors
from krcal.core.analysis_functions  import event_map

from krcal.core.analysis_functions  import kr_event_selection
from krcal.core.analysis_functions  import kr_event

from krcal.core.fit_lt_functions    import fit_lifetime
from krcal.core.fit_lt_functions    import time_fcs
from krcal.core.fit_lt_functions    import fit_fcs_in_rphi_sectors
from krcal.core.fit_lt_functions    import fit_fcs_in_xy_bin
from krcal.core.fit_lt_functions    import fit_map_rphi
from krcal.core.fit_lt_functions    import fit_map_xy
from krcal.core.fit_lt_functions    import get_time_series

from krcal.core.fit_lt_histograms   import histo_fit_sectors
from krcal.core.fit_lt_histograms   import print_fit_sectors_pars
from krcal.core.fit_lt_histograms   import plot_fit_sectors
from krcal.core.fit_lt_histograms   import plot_fit_xy
from krcal.core.fit_lt_histograms   import plot_time_fcs
from krcal.core.fit_lt_histograms   import plot_fit_lifetime_and_chi2
from krcal.core.fit_lt_histograms   import print_fit_lifetime

from krcal.core.fit_energy_functions    import fit_energy
from krcal.core.fit_energy_functions    import plot_fit_energy
from krcal.core.fit_energy_functions    import print_fit_energy
from krcal.core.fit_energy_functions    import resolution_r_z
from krcal.core.fit_energy_functions    import plot_resolution_r_z

from krcal.core.kr_types import PlotLabels, KrEvent, ASectorMap
from krcal.core.kr_types import FitType, KrSector, MapType
from krcal.core.kr_types import KrFileName
from krcal.core.kr_types import ASectorMap

from krcal.core.rphi_maps_functions   import draw_rphi_maps
from krcal.core.rphi_maps_functions   import draw_rphi_map
from krcal.core.rphi_maps_functions   import draw_rphi_maps_ts
from krcal.core.rphi_maps_functions   import draw_wedges
from krcal.core.rphi_maps_functions   import define_rphi_sectors
from krcal.core.rphi_maps_functions   import rphi_sector_map_def

from krcal.core.map_functions   import tsmap_from_fmap
from krcal.core.map_functions   import amap_from_tsmap
from krcal.core.map_functions   import relative_errors
from krcal.core.map_functions   import amap_valid_mask
from krcal.core.map_functions   import amap_valid_fraction
from krcal.core.map_functions   import amap_average
from krcal.core.map_functions   import amap_max
from krcal.core.map_functions   import amap_min
from krcal.core.map_functions   import amap_replace_nan_by_mean
from krcal.core.map_functions   import amap_replace_nan_by_zero
from krcal.core.map_functions   import regularize_maps
from krcal.core.map_functions   import get_maps_from_tsmap
from krcal.core.map_functions   import map_average
from krcal.core.map_functions   import add_mapinfo

from krcal.core.xy_maps_functions   import draw_xy_maps
from krcal.core.xy_maps_functions   import draw_xy_map
from krcal.core.xy_maps_functions   import draw_xy_maps_ts

from krcal.core.s1s2_functions  import energy_time_profile
from krcal.core.s1s2_functions  import energy_X_profile

from krcal.core.correction_functions import e0_rphi_correction
from krcal.core.correction_functions import e0_xy_correction
from krcal.core.correction_functions import lt_xy_correction
from krcal.core.correction_functions import e0_xy_correction_ts
from krcal.core.correction_functions import lt_xy_correction_ts
from krcal.core.correction_functions import lt_rphi_correction
from krcal.core.correction_functions import lt_rphi_correction_ts

import krcal.dev.corrections       as corrections 

from numpy import pi, sqrt

from typing      import List, Tuple, Dict, Sequence, Iterable

from ipywidgets import interact, interactive, fixed
from IPython.display import display

import matplotlib.pyplot as plt
from krcal.core.kr_types import KrFileName

Importing the function for the time evolution

In [2]:
from krcal.core.kr_parevol_functions import kr_time_evolution

Dst file opening and selecting kr events

In [3]:
input_path  = f"$IC_DATA/dst"
output_path = f"$IC_DATA/ldst"
map_path    = f"$IC_DATA/maps"

run_number  = 6573
input_file_names  = [f'dst_{run_number}_trigger1_0000_6679.h5']

time_bins = 25
output_file_name  = ' '
map_file_name     = ''
map_file_name_ts  = ''
emap_file_name    = f'kr_emap_xy_100_100_r_{run_number}.h5'


krfn = KrFileName(input_file_names,
                  output_file_name,
                  map_file_name,
                  map_file_name_ts,
                  emap_file_name)
fn =filenames_from_list(krfn, input_path, output_path, map_path)

In [4]:
dst           = load_dsts(fn.input_file_names, "DST", "Events")
t0            = dst.time.min()
unique_events = ~dst.event.duplicated()
number_of_evts_full = np.count_nonzero(unique_events)

print(f"Total number of events: {len(dst)}")

Total number of events: 2596957


In [5]:
dst_time = dst.sort_values('time')
T       = dst_time.time.values
DT      = time_delta_from_time(T)

In [6]:
RMAX      = 200
RCORE     = 100
s1e_range = (3, 25)
s2e_range = (2000, 14000)
s2q_range = (200, 800)

xy_range  = (-RMAX,  RMAX)
z_range   = (10,  550)
e_range = (5000, 14000)
lt_range = (1000, 6000)
c2_range = (0,5)

krTimes, krRanges, krNbins, krBins = kr_ranges_and_bins(dst,
                                                        xxrange   = xy_range,
                                                        yrange    = xy_range,
                                                        zrange    = z_range,
                                                        s2erange  = s2e_range,
                                                        s1erange  = s1e_range,
                                                        s2qrange  = s2q_range,
                                                        xnbins    = 100,
                                                        ynbins    = 100,
                                                        znbins    = 15,
                                                        s2enbins  = 25,
                                                        s1enbins  = 10,
                                                        s2qnbins  = 25,
                                                        tpsamples = 3600) # tsamples in seconds

In [7]:
dst = dst[dst.R<100]


dst_time = dst.sort_values('event')
T       = dst_time.time.values
DT      = time_delta_from_time(T)

In [8]:
kge = kr_event(dst, DT, dst.S2e, dst.S2q) 

Making time slices and list of masks

In [9]:
ti = timeit(get_time_series)
ts, masks = ti(time_bins, (DT[0],DT[-1]), kge)

Time spent in get_time_series: 0.13191008567810059 s


Reading correction map

In [10]:
emaps = read_maps(filename=fn.emap_file_name)

Applying the function for parameters evolution

In [11]:
ti = timeit(kr_time_evolution)
pars = ti(ts+t0, masks, dst, emaps, xr_map=krRanges.X, yr_map=krRanges.Y, nx_map=krNbins.X, ny_map=krNbins.X)

Time spent in kr_time_evolution: 124.6180830001831 s


In [12]:
plt.figure(figsize=(16, 20));
plt.subplot(3,2,1);
plt.title('e0');
plt.errorbar(pars.ts, pars.e0, pars.e0u, fmt='.', linestyle='-');
plt.subplot(3,2,2);
plt.title('lt');
plt.errorbar(pars.ts, pars['lt'], pars['ltu'], fmt='.', linestyle='-');
plt.subplot(3,2,3);
plt.title('dv');
#plt.ylim(0.925, 0.9275);
plt.errorbar(pars.ts, pars.dv, pars.dvu, fmt='.', linestyle='-');
plt.subplot(3,2,4);
plt.title('s1e');
plt.errorbar(pars.ts, pars.s1e, pars.s1eu, fmt='.', linestyle='-');
plt.subplot(3,2,5);
plt.title('s2e');
plt.errorbar(pars.ts, pars.s2e, pars.s2eu, fmt='.', linestyle='-');
plt.subplot(3,2,6);
plt.title('Nsipm');
plt.errorbar(pars.ts, pars.Nsipm, pars.Nsipmu, fmt='.', linestyle='-');

### Saving time parameters evolution

In [13]:
pars.to_hdf('/home/ausonandres/KrCalibNB/maps/kr_emap_xy_100_100_r_6573_time.h5', key='time_evolution', mode='a')

### Opening table again

In [14]:
test_pars = pd.read_hdf('/home/ausonandres/KrCalibNB/maps/kr_emap_xy_100_100_r_6573_time.h5', key='time_evolution')

In [15]:
test_pars

ts            e0       e0u           lt        ltu        dv  \
0   1.545134e+09  13313.485612  3.018015  3859.072899  11.510333  0.926184   
1   1.545142e+09  13322.187624  3.029075  3844.872298  11.392885  0.925967   
2   1.545149e+09  13323.593343  2.965769  3862.124487  11.348939  0.925623   
3   1.545157e+09  13322.979201  3.015349  3864.140287  11.408675  0.925998   
4   1.545164e+09  13322.820951  2.978115  3874.984835  11.420681  0.926163   
5   1.545172e+09  13327.986537  3.000973  3853.037555  11.344446  0.926627   
6   1.545179e+09  13327.644203  2.985135  3856.710995  11.282536  0.925655   
7   1.545187e+09  13329.079942  3.014723  3841.951248  11.292225  0.926051   
8   1.545195e+09  13325.481348  3.016334  3907.291035  11.718345  0.925090   
9   1.545202e+09  13327.698188  3.005562  3857.441843  11.421376  0.925545   
10  1.545210e+09  13331.905226  2.991231  3867.969978  11.269736  0.926981   
11  1.545217e+09  13326.735163  3.031034  3887.865919  11.526601  0.924993   
12  1.545225e+09  13328.323710  3.058830  3859.977553  11.616595  0.925660   
13  1.545232e+09  13329.025250  2.997490  3871.525895  11.374593  0.926448   
14  1.545240e+09  13325.979841  2.994582  3887.892908  11.581100  0.925711   
15  1.545247e+09  13328.381137  3.015467  3845.841247  11.379749  0.926493   
16  1.545255e+09  13338.742277  3.026305  3817.358468  11.235809  0.925725   
17  1.545262e+09  13335.483418  3.034876  3823.762333  11.334180  0.926683   
18  1.545270e+09  13338.882491  3.009366  3840.684341  11.279906  0.926151   
19  1.545277e+09  13335.655272  3.037392  3859.609432  11.574839  0.926188   
20  1.545285e+09  13337.895721  3.018364  3841.321120  11.390350  0.926502   
21  1.545292e+09  13337.633239  2.977604  3856.918597  11.327334  0.926221   
22  1.545300e+09  13333.763821  2.979122  3889.589282  11.467711  0.926119   
23  1.545308e+09  13339.007140  2.984235  3859.109884  11.224537  0.926308   
24  1.545315e+09  13343.635398  3.029212  3834.008171  11.381493  0.925929   
25  1.545315e+09  13343.613901  3.029260  3833.840954  11.375933  0.925938   

         dvu         s1w      s1wu       s1h    ...              s2e  \
0   0.000456  186.488744  0.420710  2.002748    ...     11426.667674   
1   0.000481  186.853677  0.424269  1.997534    ...     11428.666401   
2   0.000499  187.210896  0.422410  1.997571    ...     11436.658349   
3   0.000475  186.952367  0.422075  1.992016    ...     11438.056087   
4   0.000445  187.834862  0.422048  1.998092    ...     11438.180943   
5   0.000406  187.807922  0.423312  1.993396    ...     11435.031478   
6   0.000487  187.075293  0.420095  1.995152    ...     11437.459543   
7   0.000478  187.197828  0.420574  1.996140    ...     11432.275626   
8   0.000702  186.823061  0.423100  1.999538    ...     11448.268695   
9   0.000505  187.521944  0.425416  1.997164    ...     11437.009250   
10  0.000410  187.243144  0.422603  2.004039    ...     11442.719504   
11  0.000576  186.997368  0.423534  2.002313    ...     11434.043941   
12  0.000513  187.459863  0.428981  1.991596    ...     11438.342248   
13  0.000436  187.714871  0.424425  2.002066    ...     11436.823531   
14  0.000504  187.165555  0.422589  2.003128    ...     11445.581189   
15  0.000434  187.423122  0.426925  1.994277    ...     11440.326530   
16  0.000496  187.213585  0.424428  1.997895    ...     11439.079522   
17  0.000458  186.828035  0.421704  1.996809    ...     11440.443831   
18  0.000448  186.735706  0.419550  2.001098    ...     11443.875020   
19  0.000471  186.986847  0.428325  2.001348    ...     11445.650700   
20  0.000453  186.626493  0.421074  2.000892    ...     11450.002612   
21  0.000435  186.702398  0.420288  2.002958    ...     11448.635792   
22  0.000466  186.489936  0.420426  1.998339    ...     11450.993904   
23  0.000475  186.932637  0.423631  1.997521    ...     11437.579954   
24  0.000479  186.683246  0.422720  1.994322    ...     11447.016849   
25  0.000479  186.682440  0.422637 

### Plotting some parameters again to crosscheck

In [16]:
plt.figure(figsize=(16, 20));
plt.subplot(3,2,1);
plt.title('e0');
plt.errorbar(test_pars.ts, test_pars.e0, test_pars.e0u, fmt='.', linestyle='-');
plt.subplot(3,2,2);
plt.title('lt');
plt.errorbar(test_pars.ts, test_pars['lt'], test_pars['ltu'], fmt='.', linestyle='-');
plt.subplot(3,2,3);
plt.title('dv');
#plt.ylim(0.925, 0.9275);
plt.errorbar(test_pars.ts, test_pars.dv, test_pars.dvu, fmt='.', linestyle='-');
plt.subplot(3,2,4);
plt.title('s1e');
plt.errorbar(test_pars.ts, test_pars.s1e, test_pars.s1eu, fmt='.', linestyle='-');
plt.subplot(3,2,5);
plt.title('s2e');
plt.errorbar(test_pars.ts, test_pars.s2e, test_pars.s2eu, fmt='.', linestyle='-');
plt.subplot(3,2,6);
plt.title('Nsipm');
plt.errorbar(test_pars.ts, test_pars.Nsipm, test_pars.Nsipmu, fmt='.', linestyle='-');